In [1]:
import requests

In [2]:
service='pet-bokeh-service'
port=8500

In [3]:
requests.get(f'http://{service}:{port}/v1/models/pet-bokeh/metadata').json()

{'model_spec': {'name': 'pet-bokeh', 'signature_name': '', 'version': '1'},
 'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'Input': {'dtype': 'DT_FLOAT',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '160', 'name': ''},
         {'size': '160', 'name': ''},
         {'size': '3', 'name': ''}],
        'unknown_rank': False},
       'name': 'serving_default_Input:0'}},
     'outputs': {'Output': {'dtype': 'DT_FLOAT',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '160', 'name': ''},
         {'size': '160', 'name': ''},
         {'size': '3', 'name': ''}],
        'unknown_rank': False},
       'name': 'StatefulPartitionedCall:0'}},
     'method_name': 'tensorflow/serving/predict'},
    '__saved_model_init_op': {'inputs': {},
     'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID',
       'tensor_shape': {'dim': [], 'unknown_rank': True},
       'name': 'NoOp'}},
     'method_name':